# NLP 
    
### 1- Tokenize & StopWords 
### 2- countvectorizer for feature vectorization
### 3- VectorAssembler
### 4- Logistic Regression Model

In [99]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

import findspark
findspark.init()

import pyspark
#import SparkSession
from pyspark.sql import SparkSession

In [100]:
#create spark session
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('nlp').getOrCreate()

In [101]:
df=spark.createDataFrame([(1,'I really liked this movie'),
                         (2,'I would recommend this movie to my friends'),
                         (3,'movie was alright but acting was horrible'),
                         (4,'I am never watching that movie ever again')],
                        ['user_id','review'])

In [102]:
df.show(4,False)

+-------+------------------------------------------+
|user_id|review                                    |
+-------+------------------------------------------+
|1      |I really liked this movie                 |
|2      |I would recommend this movie to my friends|
|3      |movie was alright but acting was horrible |
|4      |I am never watching that movie ever again |
+-------+------------------------------------------+



In [103]:
# Tokenization

In [104]:
from pyspark.ml.feature import Tokenizer

In [105]:
tokenization=Tokenizer(inputCol='review',outputCol='tokens')

In [106]:
tokenized_df=tokenization.transform(df)

In [107]:
tokenized_df.show(4,False)

+-------+------------------------------------------+---------------------------------------------------+
|user_id|review                                    |tokens                                             |
+-------+------------------------------------------+---------------------------------------------------+
|1      |I really liked this movie                 |[i, really, liked, this, movie]                    |
|2      |I would recommend this movie to my friends|[i, would, recommend, this, movie, to, my, friends]|
|3      |movie was alright but acting was horrible |[movie, was, alright, but, acting, was, horrible]  |
|4      |I am never watching that movie ever again |[i, am, never, watching, that, movie, ever, again] |
+-------+------------------------------------------+---------------------------------------------------+



In [110]:
# stopwords removal 

In [111]:
from pyspark.ml.feature import StopWordsRemover

In [112]:
stopword_removal = StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [113]:
refined_df=stopword_removal.transform(tokenized_df)

In [114]:
refined_df.select(['user_id','tokens','refined_tokens']).show(10,False)

+-------+---------------------------------------------------+----------------------------------+
|user_id|tokens                                             |refined_tokens                    |
+-------+---------------------------------------------------+----------------------------------+
|1      |[i, really, liked, this, movie]                    |[really, liked, movie]            |
|2      |[i, would, recommend, this, movie, to, my, friends]|[recommend, movie, friends]       |
|3      |[movie, was, alright, but, acting, was, horrible]  |[movie, alright, acting, horrible]|
|4      |[i, am, never, watching, that, movie, ever, again] |[never, watching, movie, ever]    |
+-------+---------------------------------------------------+----------------------------------+



In [115]:
# Count Vectorizer

In [116]:
from pyspark.ml.feature import CountVectorizer

In [117]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

In [118]:
cv_df=count_vec.fit(refined_df).transform(refined_df)

In [119]:
cv_df.select(['user_id','refined_tokens','features']).show(4,False)

+-------+----------------------------------+---------------------------------+
|user_id|refined_tokens                    |features                         |
+-------+----------------------------------+---------------------------------+
|1      |[really, liked, movie]            |(11,[0,2,8],[1.0,1.0,1.0])       |
|2      |[recommend, movie, friends]       |(11,[0,4,9],[1.0,1.0,1.0])       |
|3      |[movie, alright, acting, horrible]|(11,[0,6,7,10],[1.0,1.0,1.0,1.0])|
|4      |[never, watching, movie, ever]    |(11,[0,1,3,5],[1.0,1.0,1.0,1.0]) |
+-------+----------------------------------+---------------------------------+



In [120]:
count_vec.fit(refined_df).vocabulary

['movie',
 'ever',
 'liked',
 'friends',
 'horrible',
 'alright',
 'acting',
 'recommend',
 'never',
 'really',
 'watching']

In [121]:
#Tf-idf

In [122]:
from pyspark.ml.feature import HashingTF,IDF

In [123]:
hashing_vec=HashingTF(inputCol='refined_tokens',outputCol='tf_features')

In [124]:
hashing_df=hashing_vec.transform(refined_df)

In [125]:
hashing_df.select(['user_id','refined_tokens','tf_features']).show(4,False)

+-------+----------------------------------+-------------------------------------------------------+
|user_id|refined_tokens                    |tf_features                                            |
+-------+----------------------------------+-------------------------------------------------------+
|1      |[really, liked, movie]            |(262144,[14,32675,155321],[1.0,1.0,1.0])               |
|2      |[recommend, movie, friends]       |(262144,[129613,155321,222394],[1.0,1.0,1.0])          |
|3      |[movie, alright, acting, horrible]|(262144,[80824,155321,236263,240286],[1.0,1.0,1.0,1.0])|
|4      |[never, watching, movie, ever]    |(262144,[63139,155321,203802,245806],[1.0,1.0,1.0,1.0])|
+-------+----------------------------------+-------------------------------------------------------+



In [126]:
tf_idf_vec=IDF(inputCol='tf_features',outputCol='tf_idf_features')

In [127]:
tf_idf_df=tf_idf_vec.fit(hashing_df).transform(hashing_df)

In [128]:
tf_idf_df.select(['user_id','tf_idf_features']).show(4,False)

+-------+----------------------------------------------------------------------------------------------------+
|user_id|tf_idf_features                                                                                     |
+-------+----------------------------------------------------------------------------------------------------+
|1      |(262144,[14,32675,155321],[0.9162907318741551,0.9162907318741551,0.0])                              |
|2      |(262144,[129613,155321,222394],[0.9162907318741551,0.0,0.9162907318741551])                         |
|3      |(262144,[80824,155321,236263,240286],[0.9162907318741551,0.0,0.9162907318741551,0.9162907318741551])|
|4      |(262144,[63139,155321,203802,245806],[0.9162907318741551,0.0,0.9162907318741551,0.9162907318741551])|
+-------+----------------------------------------------------------------------------------------------------+



# Classification 

In [130]:
text_df=spark.read.csv('Movie_reviews.csv',inferSchema=True,header=True,sep=',')

In [131]:
text_df.printSchema()

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [132]:
text_df.count()

7087

In [133]:
from pyspark.sql.functions import rand 

In [134]:
text_df.orderBy(rand()).show(10,False)

+---------------------------------------------------------------------+---------+
|Review                                                               |Sentiment|
+---------------------------------------------------------------------+---------+
|"I love the Harry Potter series if you can count that as "" a "" book| also"   |
|we're gonna like watch Mission Impossible or Hoot.(                  |1        |
|The Da Vinci Code is awesome!!                                       |1        |
|by the way, the Da Vinci Code sucked, just letting you know...       |0        |
|I hate Harry Potter.                                                 |0        |
|Brokeback Mountain is a beautiful film..                             |1        |
|The Da Vinci Code is awesome..                                       |1        |
|A futile mission impossible ending before it even began.             |0        |
|I LOVE THE DA VINCI CODE!!..                                         |1        |
|I love Da Vinci

In [135]:
text_df=text_df.filter(((text_df.Sentiment =='1') | (text_df.Sentiment =='0')))

In [136]:
text_df.count()

6990

In [137]:
text_df.groupBy('Sentiment').count().show()

+---------+-----+
|Sentiment|count|
+---------+-----+
|        0| 3081|
|        1| 3909|
+---------+-----+



In [138]:
text_df.printSchema()

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)



In [139]:
text_df = text_df.withColumn("Label", text_df.Sentiment.cast('float')).drop('Sentiment')

In [140]:
text_df.orderBy(rand()).show(10,False)

+------------------------------------------------------------------------+-----+
|Review                                                                  |Label|
+------------------------------------------------------------------------+-----+
|I hate Harry Potter, it's retarted, gay and stupid and there's only one |0.0  |
|Not because I hate Harry Potter, but because I am the type of person tha|0.0  |
|Harry Potter's awesome..                                                |1.0  |
|I love The Da Vinci Code...                                             |1.0  |
|Da Vinci Code sucks.                                                    |0.0  |
|i love kirsten / leah / kate escapades and mission impossible tom as wel|1.0  |
|I love Harry Potter.                                                    |1.0  |
|"Anyway, thats why I love "" Brokeback Mountain."                       |1.0  |
|I am going to start reading the Harry Potter series again because that i|1.0  |
|Brokeback Mountain is fucki

In [141]:
text_df.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  1.0| 3909|
|  0.0| 3081|
+-----+-----+



In [142]:
# Add length to the dataframe
from pyspark.sql.functions import length

In [143]:
text_df=text_df.withColumn('length',length(text_df['Review']))

In [144]:
text_df.orderBy(rand()).show(10,False)

+------------------------------------------------------------------------+-----+------+
|Review                                                                  |Label|length|
+------------------------------------------------------------------------+-----+------+
|friday hung out with kelsie and we went and saw The Da Vinci Code SUCKED|0.0  |72    |
|Mission Impossible III sucks..                                          |0.0  |30    |
|Because I like Harry Potter and I in Eragon's case, I like dragons...   |1.0  |69    |
|Combining the opinion / review from Gary and Gin Zen, The Da Vinci Code |0.0  |71    |
|The Da Vinci Code is awesome!!                                          |1.0  |30    |
|Da Vinci Code = Up, Up, Down, Down, Left, Right, Left, Right, B, A, SUCK|0.0  |72    |
|I love Brokeback Mountain....                                           |1.0  |29    |
|MI: 2 was generally agreed to be a decent action movie but a terrible fi|0.0  |72    |
|The Da Vinci Code is awesome!! 

In [145]:
text_df.groupBy('Label').agg({'Length':'mean'}).show()

+-----+-----------------+
|Label|      avg(Length)|
+-----+-----------------+
|  1.0|47.61882834484523|
|  0.0|50.95845504706264|
+-----+-----------------+



## Data Cleaning: Tokenize & StopWords

In [147]:
tokenization=Tokenizer(inputCol='Review',outputCol='tokens')

In [148]:
tokenized_df=tokenization.transform(text_df)

In [149]:
tokenized_df.show()

+--------------------+-----+------+--------------------+
|              Review|Label|length|              tokens|
+--------------------+-----+------+--------------------+
|The Da Vinci Code...|  1.0|    39|[the, da, vinci, ...|
|this was the firs...|  1.0|    72|[this, was, the, ...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|
|I liked the Da Vi...|  1.0|    72|[i, liked, the, d...|
|that's not even a...|  1.0|    72|[that's, not, eve...|
|I loved the Da Vi...|  1.0|    72|[i, loved, the, d...|
|i thought da vinc...|  1.0|    57|[i, thought, da, ...|
|The Da Vinci Code...|  1.0|    45|[the, da, vinci, ...|
|I thought the Da ...|  1.0|    51|[i, thought, the,...|
|The Da Vinci Code...|  1.0|    68|[the, da, vinci, ...|
|The Da Vinci Code...|  1.0|    62|[the, da, vinci, ...|
|then I turn on th...|  1.0|    66|[then, i, turn, o...|
|The Da Vinci Code...|  1.0|    34|[the, da, vinci, ...|
|i love da vinci c...|  1.0|   

In [150]:
stopword_removal=StopWordsRemover(inputCol='tokens',outputCol='refined_tokens')

In [151]:
refined_text_df=stopword_removal.transform(tokenized_df)

In [152]:
refined_text_df.show()

+--------------------+-----+------+--------------------+--------------------+
|              Review|Label|length|              tokens|      refined_tokens|
+--------------------+-----+------+--------------------+--------------------+
|The Da Vinci Code...|  1.0|    39|[the, da, vinci, ...|[da, vinci, code,...|
|this was the firs...|  1.0|    72|[this, was, the, ...|[first, clive, cu...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|[liked, da, vinci...|
|i liked the Da Vi...|  1.0|    32|[i, liked, the, d...|[liked, da, vinci...|
|I liked the Da Vi...|  1.0|    72|[i, liked, the, d...|[liked, da, vinci...|
|that's not even a...|  1.0|    72|[that's, not, eve...|[even, exaggerati...|
|I loved the Da Vi...|  1.0|    72|[i, loved, the, d...|[loved, da, vinci...|
|i thought da vinc...|  1.0|    57|[i, thought, da, ...|[thought, da, vin...|
|The Da Vinci Code...|  1.0|    45|[the, da, vinci, ...|[da, vinci, code,...|
|I thought the Da ...|  1.0|    51|[i, thought, the,...|[thought

In [153]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import *

In [154]:
len_udf = udf(lambda s: len(s), IntegerType())

refined_text_df = refined_text_df.withColumn("token_count", len_udf(col('refined_tokens')))


In [155]:
refined_text_df.orderBy(rand()).show(10)

+--------------------+-----+------+--------------------+--------------------+-----------+
|              Review|Label|length|              tokens|      refined_tokens|token_count|
+--------------------+-----+------+--------------------+--------------------+-----------+
|Which is why i sa...|  1.0|    72|[which, is, why, ...|[said, silent, hi...|          8|
|Mission Impossibl...|  1.0|    35|[mission, impossi...|[mission, impossi...|          4|
|I hate Harry Potter.|  0.0|    20|[i, hate, harry, ...|[hate, harry, pot...|          3|
|Brokeback Mountai...|  0.0|    40|[brokeback, mount...|[brokeback, mount...|          4|
|I would like to k...|  0.0|    72|[i, would, like, ...|[like, kill, writ...|          7|
|Because I would l...|  1.0|    72|[because, i, woul...|[like, make, frie...|          6|
|These Harry Potte...|  0.0|    38|[these, harry, po...|[harry, potter, m...|          5|
|I, too, like Harr...|  1.0|    27|[i,, too,, like, ...|[i,, too,, like, ...|          5|
|Da Vinci 

Now that we have the refined tokens after stopword removal, we can
use any of the above approaches to convert text into numerical features.
In this case, we use a **countvectorizer for feature vectorization** for the
Machine Learning Model.

In [156]:
count_vec=CountVectorizer(inputCol='refined_tokens',outputCol='features')

In [157]:
cv_text_df=count_vec.fit(refined_text_df).transform(refined_text_df)

In [158]:
cv_text_df.select(['refined_tokens','token_count','features','Label']).show(10)

+--------------------+-----------+--------------------+-----+
|      refined_tokens|token_count|            features|Label|
+--------------------+-----------+--------------------+-----+
|[da, vinci, code,...|          5|(2302,[0,1,4,43,2...|  1.0|
|[first, clive, cu...|          9|(2302,[11,51,229,...|  1.0|
|[liked, da, vinci...|          5|(2302,[0,1,4,53,3...|  1.0|
|[liked, da, vinci...|          5|(2302,[0,1,4,53,3...|  1.0|
|[liked, da, vinci...|          8|(2302,[0,1,4,53,6...|  1.0|
|[even, exaggerati...|          6|(2302,[46,229,271...|  1.0|
|[loved, da, vinci...|          8|(2302,[0,1,22,30,...|  1.0|
|[thought, da, vin...|          7|(2302,[0,1,4,228,...|  1.0|
|[da, vinci, code,...|          6|(2302,[0,1,4,33,2...|  1.0|
|[thought, da, vin...|          7|(2302,[0,1,4,223,...|  1.0|
+--------------------+-----------+--------------------+-----+
only showing top 10 rows



In [159]:
#select data for building model
model_text_df=cv_text_df.select(['features','token_count','Label'])

Use **VectorAssembler** to create input features for the machine learning model.

In [160]:
from pyspark.ml.feature import VectorAssembler

In [161]:
df_assembler = VectorAssembler(inputCols=['features','token_count'],outputCol='features_vec')
model_text_df = df_assembler.transform(model_text_df)

In [162]:
model_text_df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- token_count: integer (nullable = true)
 |-- Label: float (nullable = true)
 |-- features_vec: vector (nullable = true)



In [163]:
from pyspark.ml.classification import LogisticRegression

In [164]:
#split the data 
training_df,test_df=model_text_df.randomSplit([0.75,0.25])

In [165]:
training_df.groupBy('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|  1.0| 2867|
|  0.0| 2281|
+-----+-----+



In [166]:
test_df.groupBy('Label').count().show()

+-----+-----+
|Label|count|
+-----+-----+
|  1.0| 1042|
|  0.0|  800|
+-----+-----+



We can use any of the classification models on this data, but we
proceed with training the **Logistic Regression Model**.

In [167]:
log_reg=LogisticRegression(featuresCol='features_vec',labelCol='Label').fit(training_df)

In [168]:
results=log_reg.evaluate(test_df).predictions

In [169]:
results.show()

+--------------------+-----------+-----+--------------------+--------------------+--------------------+----------+
|            features|token_count|Label|        features_vec|       rawPrediction|         probability|prediction|
+--------------------+-----------+-----+--------------------+--------------------+--------------------+----------+
|(2302,[0,1,4,5,30...|          5|  1.0|(2303,[0,1,4,5,30...|[-22.938807722936...|[1.09094381499421...|       1.0|
|(2302,[0,1,4,5,36...|          5|  1.0|(2303,[0,1,4,5,36...|[-20.637197101780...|[1.08988123771091...|       1.0|
|(2302,[0,1,4,5,44...|          5|  1.0|(2303,[0,1,4,5,44...|[-20.829485407724...|[8.99227181577077...|       1.0|
|(2302,[0,1,4,10,1...|         10|  0.0|(2303,[0,1,4,10,1...|[32.3989845368892...|[0.99999999999999...|       0.0|
|(2302,[0,1,4,11,4...|          7|  1.0|(2303,[0,1,4,11,4...|[-23.082993660268...|[9.44459256677047...|       1.0|
|(2302,[0,1,4,12,1...|         10|  1.0|(2303,[0,1,4,12,1...|[-25.612252762764..

In [170]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator


In [171]:
#confusion matrix
true_postives = results[(results.Label == 1) & (results.prediction == 1)].count()
true_negatives = results[(results.Label == 0) & (results.prediction == 0)].count()
false_positives = results[(results.Label == 0) & (results.prediction == 1)].count()
false_negatives = results[(results.Label == 1) & (results.prediction == 0)].count()

In [172]:
recall = float(true_postives)/(true_postives + false_negatives)
print(recall)

0.9875239923224568


In [173]:
precision = float(true_postives) / (true_postives + false_positives)
print(precision)

0.9716713881019831


In [174]:
accuracy=float((true_postives+true_negatives) /(results.count()))
print(accuracy)

0.9766558089033659
